# Example for:
Transfer Learning Empirical Experiment from Vehicles (pretraining) to Animals (finetuning) classes of CIFAR

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 1
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

# device = torch.device("cuda" if torch.cuda.is_available() else "mps")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5,6]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64,
      'width':64,
      # 'hidden_dim_lin': 128,
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  # CHANGE: if no layer-wise lr
      # 'lr_fine_tune_reinit': 0.001,         # CHANGE: if no layer-wise lr
      # 'lr_fine_tune_no_reinit': 0.0001,     # CHANGE: if layer-wise lr
      'num_train': 10,
      'early_stop_patience': 5,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':64,
      # DATASET PARAMS
      'pre_train_classes': [0, 1, 8, 9],
      'fine_tune_classes': [2, 3, 4, 5, 6, 7],
        'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': True,         # CHANGE: reinit the conv lyers only after the cut
      'reinit_both_dense': True,   # CHANGE: True for reinitialize both dense layers, False for reinit only the last dense layer
      'truncate': True
    }

In [4]:
root_dir = './data'  # Specify your data directory here
transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
dataloader_wrapped = TransferLearningWrapper(params, datasets.CIFAR10, datasets.CIFAR10, root_dir, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
dataloader_wrapped.pretrain_train_loader.dataset.__len__()

18000

## Pretraining

In [6]:
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim, tuple(dataloader_wrapped.finetune_test_loader.dataset[0][0].shape))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=4, bias=True)
)

In [10]:
# Train and evaluate - Skip if loading saved model!
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 44.91%
Validation Accuracy: 43.40%
Epoch: 1 	Training Accuracy: 53.14%
Validation Accuracy: 50.33%
Epoch: 2 	Training Accuracy: 63.49%
Validation Accuracy: 60.30%
Epoch: 3 	Training Accuracy: 68.63%
Validation Accuracy: 65.00%
Epoch: 4 	Training Accuracy: 70.95%
Validation Accuracy: 67.07%
Epoch: 5 	Training Accuracy: 76.44%
Validation Accuracy: 70.37%
Epoch: 6 	Training Accuracy: 78.74%
Validation Accuracy: 71.53%
Epoch: 7 	Training Accuracy: 81.48%
Validation Accuracy: 72.73%
Epoch: 8 	Training Accuracy: 83.88%
Validation Accuracy: 73.43%
Epoch: 9 	Training Accuracy: 87.08%
Validation Accuracy: 74.83%
Final Training Accuracy: 0.8708
Final Test Accuracy: 0.8586


In [7]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_model_Vehicles_constant_channels/modelcifar10_vehicles_cpu.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=4, bias=True)
)

In [8]:
eval(pretrained_model, device, dataloader_wrapped.test_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.1832, Accuracy: 18689.0/20000 (93%)

              precision    recall  f1-score   support

     Class 0       0.91      0.95      0.93      5000
     Class 1       0.97      0.92      0.95      5000
     Class 2       0.94      0.93      0.93      5000
     Class 3       0.92      0.94      0.93      5000

    accuracy                           0.93     20000
   macro avg       0.94      0.93      0.93     20000
weighted avg       0.94      0.93      0.93     20000



0.93445

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [8]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [ ]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline

In [44]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results_jsons/baselines_freeze_{params["freeze"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [9]:
results = []
#percentages = [0.001, 0.01, 0.1, 0.5, 1.0]
percentages = [1.0]

In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Training Accuracy: 0.4454, Test Accuracy: 0.4395

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 1
Early stopping invoked.
Training Accuracy: 0.4349, Test Accuracy: 0.4304

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 2
Early stopping invoked.
Training Accuracy: 0.4304, Test Accuracy: 0.4271

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 3
Early stopping invoked.
Training Accuracy: 0.4263, Test Accuracy: 0.4225

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 4
Early stopping invoked.
Training Accuracy: 0.4396, Test Accuracy: 0.4355

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.6852, Test Accuracy: 0.6711

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 1
Early stopping invoked.
Training Accuracy: 0.5714, Test Accuracy: 0.5661

Sampled Percentage: 1.0, Samp

In [11]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.4454093371929043, 'test_acc': 0.4395}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.43490685073851404, 'test_acc': 0.4303666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.43041638833222, 'test_acc': 0.4271}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.42633414978104356, 'test_acc': 0.4225}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.43961998070214503, 'test_acc': 0.43553333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.6852222964447413, 'test_acc': 0.6710666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.5713649521264752, 'test_acc': 0.5661}, {'lr': 0.001, 'sampled_percentage': 1

In [12]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.4454093371929043, 'test_acc': 0.4395}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.43490685073851404, 'test_acc': 0.4303666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.43041638833222, 'test_acc': 0.4271}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.42633414978104356, 'test_acc': 0.4225}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.43961998070214503, 'test_acc': 0.43553333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.6852222964447413, 'test_acc': 0.6710666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.5713649521264752, 'test_acc': 0.5661}, {'lr': 0.001, 'sampled_percentage': 1

In [13]:
trunc1 = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.20153333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8333333333333334, 'test_acc': 0.216}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.20133333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.1799}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.2326}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.1975}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.21303333333333332}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.21023333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.2181}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.2033}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.625, 'test_acc': 0.21716666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.875, 'test_acc': 0.199}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.20696666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.2124}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.1879}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.7083333333333334, 'test_acc': 0.2145}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.7083333333333334, 'test_acc': 0.2104}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.9166666666666666, 'test_acc': 0.2166}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.9166666666666666, 'test_acc': 0.23183333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.18303333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.9166666666666666, 'test_acc': 0.1997}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.21176666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.2249}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.9583333333333334, 'test_acc': 0.2134}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.7083333333333334, 'test_acc': 0.1978}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.21436666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.8333333333333334, 'test_acc': 0.23366666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.21536666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9583333333333334, 'test_acc': 0.23693333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.24016666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.2211}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.21216666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.9583333333333334, 'test_acc': 0.23513333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.7916666666666666, 'test_acc': 0.22003333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.2568666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9583333333333334, 'test_acc': 0.21966666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.9166666666666666, 'test_acc': 0.2034}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.875, 'test_acc': 0.1932}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.23066666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.19353333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.75, 'test_acc': 0.23623333333333332}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.8333333333333334, 'test_acc': 0.2162}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.875, 'test_acc': 0.21683333333333332}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.19733333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.875, 'test_acc': 0.20566666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.9583333333333334, 'test_acc': 0.23556666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.9166666666666666, 'test_acc': 0.2394}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.9583333333333334, 'test_acc': 0.2187}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.7916666666666666, 'test_acc': 0.21096666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.18946666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.2474}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.26866666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.199}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.2522}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.27926666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.2621}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.2506}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.25056666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.28103333333333336}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.2805}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.2454}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.24223333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.24726666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.2713}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.2366}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.2871}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.25333333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.24983333333333332}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.2593666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.238}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.24156666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.28436666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.24456666666666665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.2734}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.2115}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.24346666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.2802}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.1936}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.2658}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.28026666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.27303333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.2565}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.25103333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.27486666666666665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.2672333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.2594666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.2733333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.2462}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.3058}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.24376666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.2942}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.2861}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.27453333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.2764666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.2324}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.2475}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.2985}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.26313333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.29606666666666664}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.22246666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.2642333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.28103333333333336}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.2088}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.28413333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.2993}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.2786}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.2398}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.2658}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.2816666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.27003333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.2786666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.28983333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.2742}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.3325}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.2593666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.30606666666666665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.2914}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.2646}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.29286666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.25566666666666665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.258}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.3149}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.2623}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.29386666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.20643333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.2556333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.2731}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.2037}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.27676666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.27436666666666665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.23066666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.2094}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.24753333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.28076666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.2611}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.2629666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.26716666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.2804}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.32353333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.25626666666666664}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.2906666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.28683333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.24686666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.26106666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.23306666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.2441}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.29323333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.2656}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.2731}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.18986666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.24106666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9583333333333334, 'test_acc': 0.23043333333333332}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.21826666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.25406666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9583333333333334, 'test_acc': 0.23526666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.23743333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.23283333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.233}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.2665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.23476666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.2437}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.2506}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.2481}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.9583333333333334, 'test_acc': 0.27026666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.23576666666666668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.9583333333333334, 'test_acc': 0.2695}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.26103333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.2653}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.2472}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.2482}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.25583333333333336}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 0.9583333333333334, 'test_acc': 0.26356666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.9583333333333334, 'test_acc': 0.2869333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.28933333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.22573333333333334}]
trunc2 = [{'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.5370370370370371, 'test_acc': 0.3077666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.7962962962962963, 'test_acc': 0.30693333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.5814814814814815, 'test_acc': 0.2942666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.6444444444444445, 'test_acc': 0.28836666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.7037037037037037, 'test_acc': 0.288}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.5296296296296297, 'test_acc': 0.2906}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.7407407407407407, 'test_acc': 0.2934}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.737037037037037, 'test_acc': 0.3106}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.5962962962962963, 'test_acc': 0.2960333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.7518518518518519, 'test_acc': 0.28773333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.4444444444444444, 'test_acc': 0.2889333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.6296296296296297, 'test_acc': 0.31926666666666664}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.7518518518518519, 'test_acc': 0.28496666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.5703703703703704, 'test_acc': 0.31776666666666664}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.5037037037037037, 'test_acc': 0.281}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.6111111111111112, 'test_acc': 0.3029}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.7703703703703704, 'test_acc': 0.2986666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.7444444444444445, 'test_acc': 0.2855}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.5962962962962963, 'test_acc': 0.30923333333333336}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.737037037037037, 'test_acc': 0.29986666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.7592592592592593, 'test_acc': 0.2948}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.7222222222222222, 'test_acc': 0.27203333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.7, 'test_acc': 0.2959}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.7481481481481481, 'test_acc': 0.30406666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.6444444444444445, 'test_acc': 0.28626666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.3656333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9444444444444444, 'test_acc': 0.36033333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.35126666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.3532666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9962962962962963, 'test_acc': 0.3569333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.3486}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.9962962962962963, 'test_acc': 0.36373333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.9962962962962963, 'test_acc': 0.37006666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.3556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.9962962962962963, 'test_acc': 0.37366666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9888888888888889, 'test_acc': 0.337}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.9888888888888889, 'test_acc': 0.3662666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.3476}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.9962962962962963, 'test_acc': 0.3859}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.3510666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.9962962962962963, 'test_acc': 0.3673666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.37233333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.33366666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.9962962962962963, 'test_acc': 0.37956666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.9925925925925926, 'test_acc': 0.3652666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.3658666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.9925925925925926, 'test_acc': 0.3505666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.3569}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.3637}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.3408}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.937037037037037, 'test_acc': 0.40423333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9962962962962963, 'test_acc': 0.3967333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9888888888888889, 'test_acc': 0.3863666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9925925925925926, 'test_acc': 0.40996666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9962962962962963, 'test_acc': 0.4077}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.9407407407407408, 'test_acc': 0.3995666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.9851851851851852, 'test_acc': 0.4048333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.9851851851851852, 'test_acc': 0.4023}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.9925925925925926, 'test_acc': 0.40113333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.9962962962962963, 'test_acc': 0.4121}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.9851851851851852, 'test_acc': 0.3900666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9888888888888889, 'test_acc': 0.4033}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9962962962962963, 'test_acc': 0.40003333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.4263}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.4006}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.41773333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.9962962962962963, 'test_acc': 0.41523333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.9962962962962963, 'test_acc': 0.3961}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.9925925925925926, 'test_acc': 0.41296666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.9925925925925926, 'test_acc': 0.4184333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.9888888888888889, 'test_acc': 0.4201}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.9814814814814815, 'test_acc': 0.40673333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.9925925925925926, 'test_acc': 0.4020666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.4099333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.9925925925925926, 'test_acc': 0.3908333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9962962962962963, 'test_acc': 0.4205333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9555555555555556, 'test_acc': 0.4137}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9481481481481482, 'test_acc': 0.4033}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.4140333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9962962962962963, 'test_acc': 0.4151}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.41813333333333336}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9962962962962963, 'test_acc': 0.42023333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.9962962962962963, 'test_acc': 0.4078}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.9962962962962963, 'test_acc': 0.40896666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.9962962962962963, 'test_acc': 0.4146}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.4002}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.4195333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.40613333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.4327666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.42443333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.9407407407407408, 'test_acc': 0.4425}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.4246666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.4123}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.42596666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.42693333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.4282}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.9962962962962963, 'test_acc': 0.4185}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.41973333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.9888888888888889, 'test_acc': 0.41323333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.9925925925925926, 'test_acc': 0.3968}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9333333333333333, 'test_acc': 0.4295333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9222222222222223, 'test_acc': 0.43693333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9481481481481482, 'test_acc': 0.42746666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9555555555555556, 'test_acc': 0.4396}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9444444444444444, 'test_acc': 0.43156666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.9555555555555556, 'test_acc': 0.4333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.937037037037037, 'test_acc': 0.44333333333333336}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.9666666666666667, 'test_acc': 0.42856666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8185185185185185, 'test_acc': 0.4237666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.9407407407407408, 'test_acc': 0.4517333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.9518518518518518, 'test_acc': 0.42443333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.9074074074074074, 'test_acc': 0.41873333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.4305333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.9518518518518518, 'test_acc': 0.451}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.9259259259259259, 'test_acc': 0.446}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.9518518518518518, 'test_acc': 0.4469666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.9592592592592593, 'test_acc': 0.45126666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.9333333333333333, 'test_acc': 0.4281}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.9444444444444444, 'test_acc': 0.4426}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.9555555555555556, 'test_acc': 0.4403}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.9444444444444444, 'test_acc': 0.43173333333333336}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 0.9333333333333333, 'test_acc': 0.44593333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 0.9259259259259259, 'test_acc': 0.4286}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.9518518518518518, 'test_acc': 0.4277666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.9444444444444444, 'test_acc': 0.42396666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9925925925925926, 'test_acc': 0.41036666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9703703703703703, 'test_acc': 0.41073333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9814814814814815, 'test_acc': 0.4029}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9925925925925926, 'test_acc': 0.42113333333333336}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9777777777777777, 'test_acc': 0.4165333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.9777777777777777, 'test_acc': 0.4064}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.9851851851851852, 'test_acc': 0.41933333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.8888888888888888, 'test_acc': 0.40326666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.8777777777777778, 'test_acc': 0.39836666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.9777777777777777, 'test_acc': 0.4318666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.9814814814814815, 'test_acc': 0.4035666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.9629629629629629, 'test_acc': 0.4017}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.9777777777777777, 'test_acc': 0.407}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.9777777777777777, 'test_acc': 0.4162}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.9851851851851852, 'test_acc': 0.4336333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.9851851851851852, 'test_acc': 0.42323333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.9925925925925926, 'test_acc': 0.42723333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.9851851851851852, 'test_acc': 0.41163333333333335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9925925925925926, 'test_acc': 0.4223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.9962962962962963, 'test_acc': 0.4178}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.9851851851851852, 'test_acc': 0.4152666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 0.9777777777777777, 'test_acc': 0.433}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 0.9814814814814815, 'test_acc': 0.4166}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.8592592592592593, 'test_acc': 0.4034333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.9888888888888889, 'test_acc': 0.40236666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.7333333333333333, 'test_acc': 0.39873333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.6925925925925925, 'test_acc': 0.3902333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.7333333333333333, 'test_acc': 0.41023333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.7222222222222222, 'test_acc': 0.3949}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.7481481481481481, 'test_acc': 0.38663333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.6851851851851852, 'test_acc': 0.40336666666666665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.7148148148148148, 'test_acc': 0.4051666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.7407407407407407, 'test_acc': 0.3973}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.7333333333333333, 'test_acc': 0.39613333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.7518518518518519, 'test_acc': 0.4157666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.7074074074074074, 'test_acc': 0.3821333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.7333333333333333, 'test_acc': 0.39366666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.725925925925926, 'test_acc': 0.39566666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.7481481481481481, 'test_acc': 0.40926666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.7037037037037037, 'test_acc': 0.4081}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.7, 'test_acc': 0.4167}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.7666666666666667, 'test_acc': 0.4086666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.7148148148148148, 'test_acc': 0.3945}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.7444444444444445, 'test_acc': 0.3974}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.7407407407407407, 'test_acc': 0.40076666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.7296296296296296, 'test_acc': 0.41286666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 0.7037037037037037, 'test_acc': 0.40063333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.7037037037037037, 'test_acc': 0.40273333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.7222222222222222, 'test_acc': 0.40423333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.725925925925926, 'test_acc': 0.4002}]
trunc3 = [{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.457037037037037, 'test_acc': 0.3718}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.45555555555555555, 'test_acc': 0.3657666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.4822222222222222, 'test_acc': 0.3699}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.5474074074074075, 'test_acc': 0.3752}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.5125925925925926, 'test_acc': 0.35786666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.4922222222222222, 'test_acc': 0.3757333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.5107407407407407, 'test_acc': 0.3678666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.49, 'test_acc': 0.3678666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.5544444444444444, 'test_acc': 0.36923333333333336}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.5622222222222222, 'test_acc': 0.38176666666666664}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.45740740740740743, 'test_acc': 0.3693666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.5177777777777778, 'test_acc': 0.36756666666666665}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.4922222222222222, 'test_acc': 0.37693333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.4451851851851852, 'test_acc': 0.3606}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.5148148148148148, 'test_acc': 0.3686333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.5396296296296297, 'test_acc': 0.3752666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.45925925925925926, 'test_acc': 0.3752}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.4507407407407407, 'test_acc': 0.3535}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.5281481481481481, 'test_acc': 0.3654}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.524074074074074, 'test_acc': 0.37466666666666665}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9811111111111112, 'test_acc': 0.5029333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.8088888888888889, 'test_acc': 0.4878}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9633333333333334, 'test_acc': 0.47983333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.8507407407407407, 'test_acc': 0.4942666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9866666666666667, 'test_acc': 0.5002666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.9792592592592593, 'test_acc': 0.505}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.9811111111111112, 'test_acc': 0.5135333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.8222222222222222, 'test_acc': 0.4855}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.9803703703703703, 'test_acc': 0.5027}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.9911111111111112, 'test_acc': 0.5164}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9755555555555555, 'test_acc': 0.5072}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.8088888888888889, 'test_acc': 0.48743333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.9540740740740741, 'test_acc': 0.49433333333333335}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.975925925925926, 'test_acc': 0.4950333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.9937037037037038, 'test_acc': 0.5121666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.9885185185185185, 'test_acc': 0.5104666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.9662962962962963, 'test_acc': 0.4995}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.9751851851851852, 'test_acc': 0.5087}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.9840740740740741, 'test_acc': 0.5092333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.7548148148148148, 'test_acc': 0.48456666666666665}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9362962962962963, 'test_acc': 0.5647}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8155555555555556, 'test_acc': 0.5542}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9348148148148148, 'test_acc': 0.5706333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9411111111111111, 'test_acc': 0.5659333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.8525925925925926, 'test_acc': 0.5516333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.8507407407407407, 'test_acc': 0.5655333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.9440740740740741, 'test_acc': 0.573}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.9318518518518518, 'test_acc': 0.5645333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.945925925925926, 'test_acc': 0.5654}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.8266666666666667, 'test_acc': 0.5625333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.932962962962963, 'test_acc': 0.5657}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9407407407407408, 'test_acc': 0.5684}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.855925925925926, 'test_acc': 0.5589666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.9322222222222222, 'test_acc': 0.566}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.8437037037037037, 'test_acc': 0.5583666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.9425925925925925, 'test_acc': 0.5671666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.945925925925926, 'test_acc': 0.5665333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.8244444444444444, 'test_acc': 0.5641}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.9396296296296296, 'test_acc': 0.5634666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.9455555555555556, 'test_acc': 0.5701333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9655555555555555, 'test_acc': 0.5715333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.847037037037037, 'test_acc': 0.5635666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9692592592592593, 'test_acc': 0.5752666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9662962962962963, 'test_acc': 0.5683333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.8762962962962964, 'test_acc': 0.5553}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.8859259259259259, 'test_acc': 0.5641666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9674074074074074, 'test_acc': 0.5710333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.8192592592592592, 'test_acc': 0.5521666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.8640740740740741, 'test_acc': 0.5609666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.8977777777777778, 'test_acc': 0.5719}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.8174074074074074, 'test_acc': 0.5575333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.797037037037037, 'test_acc': 0.5537666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.8959259259259259, 'test_acc': 0.5643333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.9674074074074074, 'test_acc': 0.5639666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.8977777777777778, 'test_acc': 0.5640666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.9729629629629629, 'test_acc': 0.5671666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.8081481481481482, 'test_acc': 0.5567}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.8751851851851852, 'test_acc': 0.5656}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.7385185185185185, 'test_acc': 0.5359666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.9740740740740741, 'test_acc': 0.5710333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.7025925925925925, 'test_acc': 0.5556666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.792962962962963, 'test_acc': 0.5640666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.7985185185185185, 'test_acc': 0.5647666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.7937037037037037, 'test_acc': 0.5633333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.7944444444444444, 'test_acc': 0.5619333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.8140740740740741, 'test_acc': 0.5777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.8011111111111111, 'test_acc': 0.5660333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.81, 'test_acc': 0.5675}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.7366666666666667, 'test_acc': 0.5674666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.8192592592592592, 'test_acc': 0.5762333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.79, 'test_acc': 0.5648333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.7814814814814814, 'test_acc': 0.5733}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8088888888888889, 'test_acc': 0.5661333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.8085185185185185, 'test_acc': 0.5714333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.8018518518518518, 'test_acc': 0.5691333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.8074074074074075, 'test_acc': 0.5642}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.7814814814814814, 'test_acc': 0.5557333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.782962962962963, 'test_acc': 0.568}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.8092592592592592, 'test_acc': 0.5663666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8074074074074075, 'test_acc': 0.5646666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8918518518518519, 'test_acc': 0.5639}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.7681481481481481, 'test_acc': 0.5495}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.8744444444444445, 'test_acc': 0.5532333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.7559259259259259, 'test_acc': 0.5512666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8759259259259259, 'test_acc': 0.5509}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.8781481481481481, 'test_acc': 0.5587333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.7607407407407407, 'test_acc': 0.5541333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.8874074074074074, 'test_acc': 0.5536333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.7803703703703704, 'test_acc': 0.5520333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.8944444444444445, 'test_acc': 0.5635}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.88, 'test_acc': 0.5573}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.8814814814814815, 'test_acc': 0.5629333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8918518518518519, 'test_acc': 0.5626}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.8703703703703703, 'test_acc': 0.5597333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.727037037037037, 'test_acc': 0.5482666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.8592592592592593, 'test_acc': 0.5501}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.6848148148148148, 'test_acc': 0.5361666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.8514814814814815, 'test_acc': 0.5570333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.8837037037037037, 'test_acc': 0.5562333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.8948148148148148, 'test_acc': 0.5595666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.64, 'test_acc': 0.5177}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.6359259259259259, 'test_acc': 0.5218666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.642962962962963, 'test_acc': 0.5234666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.6525925925925926, 'test_acc': 0.5286}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.6437037037037037, 'test_acc': 0.5270333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.6444444444444445, 'test_acc': 0.5218}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.6559259259259259, 'test_acc': 0.5286}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.6411111111111111, 'test_acc': 0.5266333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.6451851851851852, 'test_acc': 0.5281666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.6488888888888888, 'test_acc': 0.5255333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.654074074074074, 'test_acc': 0.5252333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.6462962962962963, 'test_acc': 0.5224333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.6514814814814814, 'test_acc': 0.526}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.632962962962963, 'test_acc': 0.5251666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.6537037037037037, 'test_acc': 0.5270666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.6337037037037037, 'test_acc': 0.5252666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.6462962962962963, 'test_acc': 0.5261333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.6377777777777778, 'test_acc': 0.5244666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.6507407407407407, 'test_acc': 0.5244666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.644074074074074, 'test_acc': 0.5255}]
trunc4 = [{'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.4305185185185185, 'test_acc': 0.40946666666666665}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.434, 'test_acc': 0.4055}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.44985185185185184, 'test_acc': 0.4063}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.4734074074074074, 'test_acc': 0.42433333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.4645185185185185, 'test_acc': 0.4185}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8513333333333334, 'test_acc': 0.6754666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.825925925925926, 'test_acc': 0.6517666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.8668888888888889, 'test_acc': 0.6871333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.7931111111111111, 'test_acc': 0.6524666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.8751111111111111, 'test_acc': 0.6864666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8418518518518519, 'test_acc': 0.7090666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.7788888888888889, 'test_acc': 0.6810666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.8188148148148148, 'test_acc': 0.6964333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.7331111111111112, 'test_acc': 0.6603333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.7688148148148148, 'test_acc': 0.6741333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.8401481481481482, 'test_acc': 0.6994}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8637777777777778, 'test_acc': 0.7149666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.8528148148148148, 'test_acc': 0.7078333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.8351111111111111, 'test_acc': 0.696}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.8568148148148148, 'test_acc': 0.7086}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.7263703703703703, 'test_acc': 0.6590333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.7322962962962963, 'test_acc': 0.6628}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.7082962962962963, 'test_acc': 0.6474666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.7153333333333334, 'test_acc': 0.6508333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.7228148148148148, 'test_acc': 0.6502666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.7528148148148148, 'test_acc': 0.6573333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.7117037037037037, 'test_acc': 0.6409666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.7192592592592593, 'test_acc': 0.6356}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.6953333333333334, 'test_acc': 0.6324}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.7282962962962963, 'test_acc': 0.6378}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.6305185185185185, 'test_acc': 0.5952333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.6299259259259259, 'test_acc': 0.5949333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.6240740740740741, 'test_acc': 0.5945}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.6388888888888888, 'test_acc': 0.5994333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.6347407407407407, 'test_acc': 0.5984666666666667}]
trunc5 = [{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.4454093371929043, 'test_acc': 0.4395}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.43490685073851404, 'test_acc': 0.4303666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.43041638833222, 'test_acc': 0.4271}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.42633414978104356, 'test_acc': 0.4225}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.43961998070214503, 'test_acc': 0.43553333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.6852222964447413, 'test_acc': 0.6710666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.5713649521264752, 'test_acc': 0.5661}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.7011059155347733, 'test_acc': 0.6825333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.6946485563720033, 'test_acc': 0.6777}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.8196392785571143, 'test_acc': 0.7942333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.7304238105841312, 'test_acc': 0.7195}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.7939582869442589, 'test_acc': 0.7755666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.7748830995323981, 'test_acc': 0.7582666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.7280115787129815, 'test_acc': 0.7177333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.7827506865583018, 'test_acc': 0.7663}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.7595190380761523, 'test_acc': 0.7432}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8185259407704297, 'test_acc': 0.7988666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.7857566985823499, 'test_acc': 0.7684333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.792993394195799, 'test_acc': 0.7741666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.7968900764491947, 'test_acc': 0.7786666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.6991390187782973, 'test_acc': 0.6916666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.6816967267869072, 'test_acc': 0.6749666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.6855562977807467, 'test_acc': 0.678}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.6817338380464633, 'test_acc': 0.6734}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.6655533288799822, 'test_acc': 0.6576333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.7085281674460031, 'test_acc': 0.6976666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.6702664588436131, 'test_acc': 0.6609666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.6623246492985972, 'test_acc': 0.6533}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.6924961033177466, 'test_acc': 0.6824}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.6710829065538484, 'test_acc': 0.6643666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.6209827061530468, 'test_acc': 0.6174}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.6173829139761003, 'test_acc': 0.6133333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.6166778000445335, 'test_acc': 0.6135666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.6193869219921324, 'test_acc': 0.6155333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.6212424849699398, 'test_acc': 0.6173666666666666}]

In [14]:
results = trunc1 + trunc2 + trunc3 + trunc4 + trunc5

In [ ]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results_jsons/results_freeze_{params["freeze"]}_reinit_{params["reinit"]}_pool_{params["use_pooling"]}_lr_{params["lr_fine_tune"]}_nice_curve.json', 'w') as f:
    json.dump(results, f)
results = results[1:]

In [15]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cifarstyle_results_cifar_truncation.json', 'w') as f:
    json.dump(results, f)
results = results[1:]